In [68]:
import pandas as pd
from pprint import pprint
from collections import defaultdict

In [35]:
uri = '/Users/david.hedges/Documents/woodahl_lab/pgx_tool/pgx/Star Allele Dictionary.xlsx'
df = pd.read_excel(uri, na_filter=None)
#df

In [77]:
df = df.replace({'-':''}, regex=True)

changes = ['change_1','change_2','change_3','change_4','change_5',
           'change_6','change_7','change_8','change_9','change_10',
           'change_11','change_12','change_13','change_14','change_15',
           'change_16','change_17','change_18','change_19','change_20',
           'change_21','change_22','change_23','change_24','change_25',
           'change_26','change_27','change_28','change_29','change_30',
           'change_31','change_32','change_33','change_34','change_35',
           'change_36','change_37','change_38']

for i in changes:
    df[i] = df[i].str.replace(r"\(.*\)","")
    
df

,Allele,mtID,change_1,change_2,change_3,change_4,change_5,change_6,change_7,change_8,...,change_29,change_30,change_31,change_32,change_33,change_34,change_35,change_36,change_37,change_38
0,CYP2A13*1.001,,,,,,,,,,...,,,,,,,,,,
1,CYP2A13*1.002,,12C>A,,,,,,,,...,,,,,,,,,,
2,CYP2A13*1.003,,187T>C,,,,,,,,...,,,,,,,,,,
3,CYP2A13*1.004,,1757A>G,,,,,,,,...,,,,,,,,,,
4,CYP2A13*1.005,,1894G>T,,,,,,,,...,,,,,,,,,,
5,CYP2A13*1.006,,6461T>A,,,,,,,,...,,,,,,,,,,
6,CYP2A13*1.007,,7571G>C,,,,,,,,...,,,,,,,,,,
7,CYP2A13*1.008,,1479T>C,1240A>G,411G>A,1757A>G,2366C>T,6424C>T,6432C>T,7233T>G,...,,,,,,,,,,
8,CYP2A13*1.009,,523C>T,,,,,,,,...,,,,,,,,,,
9,CYP2A13*1.010,,1991C>A,,,,,,,,...,,,,,,,,,,


Need to convert transcript location to genomic location

make it compatible with both build 37 and 38

Plan:
- use try/except to parse the 12C>A (or 1634_1635insACC or substitution) type string into integers and the bases.
- regular expressions!!
- match the start from the transcript location to the genomic location
- wave magic wand and get the mtIDs for each variant (not just each haplotype)
- correlate each haplotype with an mtID profile

In [37]:
# Start locations for genes
genes38 = {'CYP2A13':41088451,'CYP2C8':95036772,'CYP2C9':94938658,'CYP2C19':94762681,
    'CYP2D6':42126499,'CYP2F1':41114292,'CYP2J2':59893308,'CYP2R1':14878005,
    'CYP2S1':41193219,'CYP2W1':983155,'CYP3A7':99705036,'CYP3A43':99827626,
    'CYP4F2':15878023,'CYP26A1':93073475,'NUDT14':105172939}

genes37 = {'CYP2A13':41594356,'CYP2C8':96796529,'CYP2C9':96698350,'CYP2C19':96522463,
    'CYP2D6':42522501,'CYP2F1':41620312,'CYP2J2':60358980,'CYP2R1':14899551,
    'CYP2S1':41699115,'CYP2W1':1020100,'CYP3A7':99302660,'CYP3A43':99425636,
    'CYP4F2':15988834,'CYP26A1':94833232,'NUDT14':105639274}

chroms = {'CYP2A13':19,'CYP2C8':10,'CYP2C9':10,'CYP2C19':10,
    'CYP2D6':22,'CYP2F1':19,'CYP2J2':1,'CYP2R1':11,
    'CYP2S1':19,'CYP2W1':7,'CYP3A7':7,'CYP3A43':7,
    'CYP4F2':19,'CYP26A1':10,'NUDT14':14}

In [38]:
# pop out mtID from dataframe
df_change = df.drop(['mtID'], axis=1)
#print(df)

In [54]:
# replace allele name with gene name
df_change['Allele'] = df_change['Allele'].map(lambda x: x.split('*',1)[0])
#print(df_change)
df_change.shape

(706, 39)

In [52]:
# make list (set) of all variants within the gene
#variant_dict = df_change.set_index('Allele').T.to_dict('list')
variant_dict = df_change.T.to_dict('list')
len(variant_dict)

706

In [67]:
# remove empty strings and clean whitespace
for key, val in variant_dict.items():
    #key.strip()
    val[:] = [i.strip() for i in val if i != '']

variant_dict

{0: ['CYP2A13'],
 1: ['CYP2A13', '12C>A'],
 2: ['CYP2A13', '187T>C'],
 3: ['CYP2A13', '1757A>G'],
 4: ['CYP2A13', '1894G>T'],
 5: ['CYP2A13', '6461T>A'],
 6: ['CYP2A13', '7571G>C'],
 7: ['CYP2A13',
  '1479T>C',
  '1240A>G',
  '411G>A',
  '1757A>G',
  '2366C>T',
  '6424C>T',
  '6432C>T',
  '7233T>G',
  '7520C>G'],
 8: ['CYP2A13', '523C>T'],
 9: ['CYP2A13', '1991C>A'],
 10: ['CYP2A13', '7365T>C'],
 11: ['CYP2A13',
  '1479T>C',
  '1429A>G',
  '1240A>G',
  '411G>A',
  '74G>A',
  '1757A>G',
  '2211T>C',
  '3375C>T'],
 12: ['CYP2A13',
  '1479T>C',
  '1429A>G',
  '1240A>G',
  '411G>A',
  '74G>A',
  '1757A>G',
  '2211T>C',
  '3375C>T',
  '6424C>T',
  '6432C>T',
  '7233T>G',
  '7571G>C'],
 13: ['CYP2A13',
  '1479T>C',
  '729C>T',
  '411G>A',
  '1634_1635insACC',
  '1706C>G',
  '1757A>G',
  '1801G>C',
  '1970A>C',
  '2211T>C',
  '5530T>C',
  '5578A>C',
  '6424C>T',
  '7233T>G',
  '7520C>G'],
 14: ['CYP2A13', '579G>A'],
 15: ['CYP2A13', '7343T>A'],
 16: ['CYP2A13', '7465C>T'],
 17: ['CYP2A13', '5

In [82]:
# create master list of variants for each gene
genes = set([val[0] for key,val in variant_dict.items()])

dd_variants = defaultdict(list)
for k,v in variant_dict.items():
    dd_variants[v[0]].append(v[1:])
    
for k,v in dd_variants.items():
    dd_variants[k] = set([i for j in v for i in j])
    
dd_variants

defaultdict(list,
            {'CYP2A13': {'1240A>G',
              '12C>A',
              '1429A>G',
              '1479T>C',
              '1634_1635insACC',
              '1706C>G',
              '1757A>G',
              '1801G>C',
              '187T>C',
              '1894G>T',
              '1970A>C',
              '1991C>A',
              '2211T>C',
              '2366C>T',
              '3375C>T',
              '411G>A',
              '523C>T',
              '5294G>T',
              '5530T>C',
              '5578A>C',
              '578C>T',
              '5792T>C',
              '579G>A',
              '6424C>T',
              '6432C>T',
              '6461T>A',
              '7233T>G',
              '729C>T',
              '7343T>A',
              '7365T>C',
              '7465C>T',
              '74G>A',
              '7520C>G',
              '7571G>C'},
             'CYP2C8': {'10918T>G',
              '10961G>C',
              '10989A>G',
              '11041C>G',
        

In [24]:
# convert all variants to genomic position

# make mtID_dict (mtID37 and mtID38)
# mtID_dict = [chrom#, variant_type, genomic_pos, ref, alt]

# wrap this in a function of functions:
# one func each gets chrom#, var_type, gen_pos, ref, alt

def genomic_postion():
    #look up gene in dict genes38 or genes37
    #set genomic start pos as variable
    #use regex to extract gene position from 1247C>A, save as variable
    #add genomics start pos to gene pos, return it

def chrom_num():
    #look up gene in chroms, return it
    
def var_type():
    #what are all the options?
    #if __ contains >,ins,del
    #return it
    
def ref():
    #look up base at genomic pos for indels
    #look up base in 1247C>A if > in value
    #return it
    
def alt():
    #parallel ref()
    #return it

In [64]:
dic={}
mydict = {
    'a':[1,2,3],
    'a':[4,5,6],
    'b':[1,2,3],
    'b':[4,5,6]}

for k,v in mydict.items():
    dic.setdefault(k,[]).append(v)
    
dic

{'a': None, 'b': None}

In [63]:
help(dict.setdefault)

Help on method_descriptor:

setdefault(self, key, default=None, /)
    Insert key with a value of default if key is not in the dictionary.
    
    Return the value for key if key is in the dictionary, else default.



In [72]:
mydict = {
    1:['a',1,2,3],
    2:['a',4,5,6],
    3:['b',1,2,3],
    4:['b',4,5,6]}

ddic = defaultdict(list)
for k,v in mydict.items():
    ddic[v[0]].append(v[1:])
    
for k,v in ddic.items():
    ddic[k] = [i for j in v for i in j]

In [73]:
ddic

defaultdict(list, {'a': [1, 2, 3, 4, 5, 6], 'b': [1, 2, 3, 4, 5, 6]})